In [1]:
import torch

print("버전:", torch.__version__)
print("GPU 사용 가능:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))


버전: 2.5.1+cu121
GPU 사용 가능: True
GPU 이름: NVIDIA GeForce RTX 2070 SUPER


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import time
from IPython import display
import imageio
import glob
import torch
import torch.optim as optim
import numpy as np

# CIFAR-10 preprocessing
cifar_transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=cifar_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=cifar_transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print("Train 개수:", len(train_dataset))
print("Test 개수:", len(test_dataset))

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class Generator(nn.Module):
    def __init__(self, z_dim=100):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, 1, 0, bias=False),   # z: (B, 100, 1, 1)
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),     # (B, 256, 8, 8)
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),     # (B, 128, 16, 16)
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),      # (B, 64, 32, 32)
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),        # (B, 3, 64, 64)
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),      # 64x64 → 32x32
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),    # 32x32 → 16x16
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),   # 16x16 → 8x8
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),   # 8x8 → 4x4
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
)

    def forward(self, x):
        return self.model(x)


# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cifar_generator = Generator().to(device)
cifar_generator.apply(weights_init)
cifar_discriminator = Discriminator().to(device)
cifar_discriminator.apply(weights_init)

criterion = nn.BCEWithLogitsLoss()
gen_optimizer = optim.Adam(cifar_generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
dis_optimizer = optim.Adam(cifar_discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Discriminator가 너무 강해지는 걸 막기 위해 label smoothing 추가
def generator_loss(fake_output, criterion):
    return criterion(fake_output, torch.ones_like(fake_output))  # 약한 정답 사용

def discriminator_loss(real_output, fake_output, criterion):
    real_labels = torch.full_like(real_output, 0.9)  # soft label
    fake_labels = torch.zeros_like(fake_output)
    real_loss = criterion(real_output, real_labels)
    fake_loss = criterion(fake_output, fake_labels)
    return real_loss + fake_loss

def discriminator_accuracy(real_output, fake_output):
    real_acc = (real_output >= 0.5).float().mean().item()
    fake_acc = (fake_output < 0.5).float().mean().item()
    return real_acc, fake_acc

noise_dim = 100
fixed_noise = torch.randn(64, noise_dim, device=device)

def train_step(real_images, generator, discriminator, gen_optimizer, dis_optimizer, criterion, device):
    batch_size = real_images.size(0)
    noise = torch.randn(batch_size, noise_dim, 1, 1, device=device)

    # Discriminator update
    generator.eval()
    discriminator.train()

    with torch.no_grad():
        fake_images = generator(noise)

    real_output = discriminator(real_images)
    fake_output = discriminator(fake_images)
    dis_loss = discriminator_loss(real_output, fake_output, criterion)

    dis_optimizer.zero_grad()
    dis_loss.backward()
    dis_optimizer.step()

    # Generator update
    generator.train()
    discriminator.eval()

    noise = torch.randn(batch_size, 100, 1, 1, device=device)
    fake_images = generator(noise)
    fake_output = discriminator(fake_images)
    gen_loss = generator_loss(fake_output, criterion)

    gen_optimizer.zero_grad()
    gen_loss.backward()
    gen_optimizer.step()

    real_acc, fake_acc = discriminator_accuracy(real_output, fake_output)
    return gen_loss.item(), dis_loss.item(), real_acc, fake_acc

def generate_and_save_images(model, epoch, it, sample_seeds, save_dir):
    model.eval()
    with torch.no_grad():
        predictions = model(sample_seeds.to(next(model.parameters()).device))
    predictions = predictions.cpu().numpy()
    fig = plt.figure(figsize=(8, 8))
    for i in range(predictions.shape[0]):
        img = predictions[i]
        img = (img + 1) / 2
        img = np.transpose(img, (1, 2, 0))
        plt.subplot(8, 8, i + 1)
        plt.imshow(img)
        plt.axis('off')
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(f'{save_dir}/sample_epoch_{epoch:04d}_iter_{it:03d}.png')
    plt.close(fig)

def draw_train_history(history, epoch, save_dir):
    plt.figure(figsize=(15, 6))
    plt.subplot(211)
    plt.plot(history['gen_loss'], label='gen_loss')
    plt.plot(history['disc_loss'], label='disc_loss')
    plt.legend()
    plt.title('Model Loss')
    plt.xlabel('Batch iters')
    plt.ylabel('Loss')

    plt.subplot(212)
    plt.plot(history['real_accuracy'], label='real_accuracy')
    plt.plot(history['fake_accuracy'], label='fake_accuracy')
    plt.legend()
    plt.title('Discriminator Accuracy')
    plt.xlabel('Batch iters')
    plt.ylabel('Accuracy')

    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(f'{save_dir}/train_history_{epoch:04d}.png')
    plt.close()

def train(dataset, generator, discriminator, gen_optimizer, dis_optimizer,
          criterion, device, epochs=50, save_every=5, sample_seeds=None):
    history = {'gen_loss':[], 'disc_loss':[], 'real_accuracy':[], 'fake_accuracy':[]}
    start = time.time()

    for epoch in range(epochs):
        epoch_start = time.time()
        for it, (real_images, _) in enumerate(dataset):
            real_images = real_images.to(device)
            gen_loss, dis_loss, real_acc, fake_acc = train_step(
                real_images, generator, discriminator,
                gen_optimizer, dis_optimizer, criterion, device
            )
            history['gen_loss'].append(gen_loss)
            history['disc_loss'].append(dis_loss)
            history['real_accuracy'].append(real_acc)
            history['fake_accuracy'].append(fake_acc)

            if it % 50 == 0:
                display.clear_output(wait=True)
                generate_and_save_images(generator, epoch+1, it+1, sample_seeds, './CIFAR-10/cifar_samples')
                print(f"Epoch {epoch+1} | Iter {it+1}")
                print(f"Epoch Time: {int(time.time() - epoch_start)} sec")

        checkpoint_dir = './CIFAR-10/cifar_training_checkpoints'
        os.makedirs(checkpoint_dir, exist_ok=True)

        if (epoch + 1) % save_every == 0:
            torch.save(generator.state_dict(), f'{checkpoint_dir}/generator_epoch_{epoch+1}.pth')
            torch.save(discriminator.state_dict(), f'{checkpoint_dir}/discriminator_epoch_{epoch+1}.pth')

        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch+1, it+1, sample_seeds, './CIFAR-10/cifar_samples')
        draw_train_history(history, epoch+1, './CIFAR-10/cifar_training_history')
        print(f"Epoch {epoch+1} completed in {int(time.time() - epoch_start)} sec")

    print(f"Training completed in {int(time.time() - start)} sec")

fixed_noise = torch.randn(64, 100, 1, 1, device=device)
train(train_loader, cifar_generator, cifar_discriminator,
      gen_optimizer, dis_optimizer, criterion,
      device, epochs=50, save_every=5, sample_seeds=fixed_noise)

Epoch 1 | Iter 351
Epoch Time: 68 sec


KeyboardInterrupt: 

In [174]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import time
from IPython import display
import imageio
import glob
import torch
import torch.optim as optim
import numpy as np
import torchvision


# CIFAR-10 preprocessing
cifar_transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=cifar_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=cifar_transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print("Train 개수:", len(train_dataset))
print("Test 개수:", len(test_dataset))

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class Generator(nn.Module):
    def __init__(self, z_dim=100):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, 1, 0, bias=False),   # z: (B, 100, 1, 1)
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),     # (B, 256, 8, 8)
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),     # (B, 128, 16, 16)
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),      # (B, 64, 32, 32)
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),        # (B, 3, 64, 64)
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),      # 64x64 → 32x32
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),    # 32x32 → 16x16
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),   # 16x16 → 8x8
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),   # 8x8 → 4x4
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
)

    def forward(self, x):
        return self.model(x)


# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cifar_generator = Generator().to(device)
cifar_generator.apply(weights_init)
cifar_discriminator = Discriminator().to(device)
cifar_discriminator.apply(weights_init)

criterion = nn.BCELoss()
gen_optimizer = optim.Adam(cifar_generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
dis_optimizer = optim.Adam(cifar_discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# 고정 시드로 생성할 노이즈
fixed_noise = torch.randn(64, 100, 1, 1, device=device)

# 학습 루프
epochs = 25
for epoch in range(epochs):
    for i, (real_images, _) in enumerate(train_loader):
        batch_size = real_images.size(0)
        real_images = real_images.to(device)

        # 진짜/가짜 라벨
        real_labels = torch.full((batch_size,), 1.0, device=device)
        fake_labels = torch.full((batch_size,), 0.0, device=device)

        #### 1. Discriminator 학습 ####
        cifar_discriminator.zero_grad()

        # 진짜 이미지
        output_real = cifar_discriminator(real_images).view(-1)
        loss_real = criterion(output_real, real_labels)

        # 가짜 이미지
        noise = torch.randn(batch_size, 100, 1, 1, device=device)
        fake_images = cifar_generator(noise)
        output_fake = cifar_discriminator(fake_images.detach()).view(-1)
        loss_fake = criterion(output_fake, fake_labels)

        # 총 loss 계산 및 역전파
        loss_D = loss_real + loss_fake
        loss_D.backward()
        dis_optimizer.step()

        #### 2. Generator 학습 ####
        cifar_generator.zero_grad()
        output = cifar_discriminator(fake_images).view(-1)
        loss_G = criterion(output, real_labels)  # Generator는 진짜처럼 보이길 원함
        loss_G.backward()
        gen_optimizer.step()

        if i % 100 == 0:
            print(f"[{epoch}/{epochs}][{i}/{len(train_loader)}] Loss_D: {loss_D.item():.4f} Loss_G: {loss_G.item():.4f}")

    # 에폭 끝날 때 이미지 저장
    with torch.no_grad():
        fake = cifar_generator(fixed_noise).detach().cpu()
    os.makedirs("cifar_samples", exist_ok=True)
    torchvision.utils.save_image(fake, f"cifar_samples/fake_epoch_{epoch:03d}.png", normalize=True)

print("✅ 학습 완료")

Files already downloaded and verified
Files already downloaded and verified
Train 개수: 50000
Test 개수: 10000
[0/25][0/391] Loss_D: 1.7231 Loss_G: 3.0261
[0/25][100/391] Loss_D: 1.3020 Loss_G: 25.0515
[0/25][200/391] Loss_D: 0.4529 Loss_G: 5.6917
[0/25][300/391] Loss_D: 0.0420 Loss_G: 6.1406
[1/25][0/391] Loss_D: 0.6738 Loss_G: 5.0536
[1/25][100/391] Loss_D: 1.9142 Loss_G: 7.6141
[1/25][200/391] Loss_D: 0.1519 Loss_G: 4.1469
[1/25][300/391] Loss_D: 0.6503 Loss_G: 6.2011
[2/25][0/391] Loss_D: 0.5857 Loss_G: 3.0665
[2/25][100/391] Loss_D: 0.4523 Loss_G: 3.5095
[2/25][200/391] Loss_D: 0.8105 Loss_G: 2.7295
[2/25][300/391] Loss_D: 0.4686 Loss_G: 3.7898
[3/25][0/391] Loss_D: 0.2508 Loss_G: 3.6535
[3/25][100/391] Loss_D: 1.3923 Loss_G: 8.5584
[3/25][200/391] Loss_D: 0.6312 Loss_G: 4.9120
[3/25][300/391] Loss_D: 0.3110 Loss_G: 4.0873
[4/25][0/391] Loss_D: 0.5139 Loss_G: 3.7690
[4/25][100/391] Loss_D: 0.4185 Loss_G: 3.4644
[4/25][200/391] Loss_D: 0.3504 Loss_G: 3.1024
[4/25][300/391] Loss_D: 0.41

### 회고
- 32 * 32 이미지, batch = 256, 32 * 32 이미지, batch = 128, 64 * 64 이미지, batch = 128 등 다양한 시도를 하였으나 형상이 나타나지 않은 문제가 발생함.
- 구글에 있는 코드(test.ipynb)를 사용해서 돌려보니 비슷하지는 않지만, 이미지 생성이 되는 것을 확인함.
- lms 기반 코드와 구글에 있는 코드를 비교하면서 최대한 맞추려고 했으나 문제를 해결하지 못함.